# Application Review - Performance Test

### [MFG] - [NAME] - [MODEL] - [POWER]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import nptdms as td
from scripts.wpphist import plot as hist_plot
from scripts.vib_files import getTdmsFilesInPath, getTdmsFilesInFolder
from scripts.driveTdms import filterDataFrame
%matplotlib inline

### Constants from Performance Curve and Initial Notebook Setup

In [ ]:
perf_curve_cac_min = 43  #degrees C
perf_curve_cac_max = 52  #degrees C

perf_curve_max_top_tank = 110 #degrees C

perf_curve_min_water_pump_inlet_95C = 30 #kPa

perf_curve_max_intake_restriction = -3.75 #kPa

perf_curve_min_exhaust_backpressure = 4 #kPa
perf_curve_max_exhaust_backpressure = 10 #kPa

HIDE_EXTRA_OUTPUT = True #reduce ammount of output in notebook
FILTER_NAN = True  #remove columns from TDMS data where all values are not a number or all the same number

#Find all tdms files in this path
mypath = './'
result = getTdmsFilesInFolder(mypath)  #only return files in mypath
#result = getTdmsFilesInPath(mypath)   #return files in mypath and all subfolders
print("Index:\t\tFile:")
[print("result[{}]\t{}".format(index, f)) for index, f in enumerate(result)]
print ('{0} files found'.format(len(result)))

### Specify TDMS file and Setup Data Structures

In [ ]:
#Can specify directly or with index
#tdfile = "./data/sample.tdms"
tdfile = result[1]

#Updated npTDMS to Pandas conversion
print(tdfile)
tdms_file = td.TdmsFile.read(tdfile)
print(tdms_file.properties)

#create Pandas dataframe and mark it as unmodified from the TDMS channel names
df = tdms_file.as_dataframe()
#whenever this cell runs the df object is re-instantiated.  Flag the new dataframe as unfiltered
FILTERED = False

### Read Available Data Headings from TDMS file specied

In [ ]:
df, FILTERED = filterDataFrame(df, FILTERED, FILTER_NAN)
if not HIDE_EXTRA_OUTPUT:
    all_groups = tdms_file.groups()
    for group in all_groups:
        print(group)
        print(group.properties)
        for channel in group.channels():
            print(channel.properties)
if not HIDE_EXTRA_OUTPUT:
    print("All Available Fields:\n")
    [print(field) for field in df.columns.values]
else:
    print("Available Fields (CAN Channels not listed):\n")
    [print(field) for field in df.columns.values if not field.startswith('C')]

# Specify aggregate calculations

In [ ]:
#General Calculations
df['Ambient'] = (df.T01_AmbientAir_01) # + df.T02_AmbientAir_02) / 2.0
df['LAT'] = perf_curve_max_top_tank - df.C22_EngineCoolantTemp + df.Ambient
df['IntakeTempRise'] = df.T04_Turbo_Air_Inlet - df.Ambient
#df['CorrectedCAC'] = 25 - df.Ambient + df.C23_EngineCACOutletTemp
#df['IntakeRestrictionAvailable'] = df.P01_Intake_Air_Restriction - perf_curve_max_intake_restriction
#Test Specific Calculations

### Plot overall Run.  Look at index at bottom for more detailed plot in next cell.

In [ ]:
df.plot(figsize=(15,45), color=('r'), subplots=True );

# Place the start and end timestamp of the test to be analyzed below

Not all data in the data frame should go into the CAC and temp rise histograms.  Specify only the data where the engine has been warmed up and is running under a representative load for this subset of the data.  Full test runs can be obtained by referening the df data frame instead of this test_subset data frame

In [ ]:
analysis_start = 200
analysis_end = 1000

## Use the analysis start and end times to create a slice of the data named test_subset which should be fed into the calcualtions

In [ ]:
assert(analysis_start < analysis_end)
test_subset = df[analysis_start:analysis_end]  #test_subset = df[800:1550] #can just specify range here

section_plot = test_subset.plot(figsize=(15,30), color = ('g'), subplots=True);

# Histograms from the test_subset data

In [ ]:
hist_plot(data=test_subset['IntakeTempRise'].values, ylabel='Count', xlabel='Temperature (degC)', 
             title='Intake Temp Rise (degC) Histogram', color='green')

In [ ]:
hist_plot(data=test_subset['LAT'].values, ylabel='Count', xlabel='Temperature (degC)', 
             title='Limiting Ambient Temperature (degC) Histogram', color='orange')

In [ ]:
#hist_plot(data=test_subset['CorrectedCAC'].values, ylabel='Count', xlabel='Temperature (degC)', 
#             title='Corrected CAC (degC) Histogram', color='blue')

In [ ]:
#hist_plot(data=test_subset['IntakeRestrictionAvailable'].values, ylabel='Count', xlabel='Pressure (kPa)', 
#             title='Intake Restriction Margin (kPa) Histogram', color='pink')